In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
# Generate our categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df.APPLICATION_TYPE.value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than 500
replace_application = list(application_type_count[application_type_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
CLASSIFICATION_count = application_df.CLASSIFICATION.value_counts()
CLASSIFICATION_count.head

<bound method NDFrame.head of C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2170        1
C2570        1
C1570        1
C1245        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64>

In [7]:
# Determine which values to replace if counts are less than 1000
replace_class = list(CLASSIFICATION_count[CLASSIFICATION_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Optimization procedure #1, Add 1 additional hidden layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Create a callback that saves the model's weights
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [15]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=200,validation_data=(X_test, y_test),callbacks=[cp_callback])


Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 27284.5104 - accuracy: 0.5079 - val_loss: 24293.5273 - val_accuracy: 0.4668

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 23299.4339 - accuracy: 0.4757 - val_loss: 12982.1416 - val_accuracy: 0.4668

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 25136.2155 - accuracy: 0.4931 - val_loss: 1261.8617 - val_accuracy: 0.4668

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 6042.1791 - accuracy: 0.5095 - val_loss: 1542.2799 - val_accuracy: 0.4668

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 2294.4680 - accuracy: 0.4991 - val_loss: 1038.1901 - val_accuracy: 0.4668

Epoch 00005: saving model to training_1\cp.ckpt
Epoch

804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5340 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00043: saving model to training_1\cp.ckpt
Epoch 44/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5281 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00044: saving model to training_1\cp.ckpt
Epoch 45/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5281 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00045: saving model to training_1\cp.ckpt
Epoch 46/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6908 - accuracy: 0.5350 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00046: saving model to training_1\cp.ckpt
Epoch 47/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6908 - accuracy: 0.5349 - val_loss: 0.6919 - val_accuracy: 0.5332

Epoch 00047: saving model to training_1\cp.ckpt
Epoch 48/200
804/804 [==========================


Epoch 00085: saving model to training_1\cp.ckpt
Epoch 86/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6904 - accuracy: 0.5379 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00086: saving model to training_1\cp.ckpt
Epoch 87/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5307 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00087: saving model to training_1\cp.ckpt
Epoch 88/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5286 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00088: saving model to training_1\cp.ckpt
Epoch 89/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5320 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00089: saving model to training_1\cp.ckpt
Epoch 90/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5342 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00090: saving model to train

804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5336 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00128: saving model to training_1\cp.ckpt
Epoch 129/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5341 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00129: saving model to training_1\cp.ckpt
Epoch 130/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5282 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00130: saving model to training_1\cp.ckpt
Epoch 131/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5310 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00131: saving model to training_1\cp.ckpt
Epoch 132/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6903 - accuracy: 0.5384 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00132: saving model to training_1\cp.ckpt
Epoch 133/200
804/804 [=====================

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5322 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00170: saving model to training_1\cp.ckpt
Epoch 171/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6906 - accuracy: 0.5363 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00171: saving model to training_1\cp.ckpt
Epoch 172/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6905 - accuracy: 0.5379 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00172: saving model to training_1\cp.ckpt
Epoch 173/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5294 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00173: saving model to training_1\cp.ckpt
Epoch 174/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5330 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00174: saving model to training_1\cp.ckpt
Epoch 175/200
804/804 [=====================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6903 - accuracy: 0.5380
Loss: 0.6903000473976135, Accuracy: 0.5379591584205627


In [17]:
# Optimization procedure #2, Alter input data by changing shape of APPLICATION_TYPE
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [19]:
# Determine which values to replace if counts are less than 1000
replace_application = list(application_type_count[application_type_count < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Determine which values to replace if counts are less than 1000
replace_class = list(CLASSIFICATION_count[CLASSIFICATION_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [21]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 80)                3280      
_________________________________________________________________
dense_5 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Create a callback that saves the model's weights
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=5)

In [28]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=150,validation_data=(X_test, y_test),callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 44664.0659 - accuracy: 0.4869 - val_loss: 342780.2812 - val_accuracy: 0.5332

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 97248.3939 - accuracy: 0.5195 - val_loss: 12656.0029 - val_accuracy: 0.4668

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 12403.4945 - accuracy: 0.5005 - val_loss: 874.5938 - val_accuracy: 0.4668

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 48336.1830 - accuracy: 0.5124 - val_loss: 13195.8564 - val_accuracy: 0.4668

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 29732.1714 - accuracy: 0.4766 - val_loss: 3069.0659 - val_accuracy: 0.4668

Epoch 00005: saving model to training_1\cp.ckpt
Ep

Epoch 43/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6906 - accuracy: 0.5371 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00043: saving model to training_1\cp.ckpt
Epoch 44/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00044: saving model to training_1\cp.ckpt
Epoch 45/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5298 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00045: saving model to training_1\cp.ckpt
Epoch 46/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5325 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00046: saving model to training_1\cp.ckpt
Epoch 47/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5279 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00047: saving model to training_1\cp.ckpt
Epoch 48/150
804/804 [=============

804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5320 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00085: saving model to training_1\cp.ckpt
Epoch 86/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6906 - accuracy: 0.5367 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00086: saving model to training_1\cp.ckpt
Epoch 87/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5320 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00087: saving model to training_1\cp.ckpt
Epoch 88/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5279 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00088: saving model to training_1\cp.ckpt
Epoch 89/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5323 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00089: saving model to training_1\cp.ckpt
Epoch 90/150
804/804 [==========================

804/804 [==============================] - 1s 1ms/step - loss: 0.6919 - accuracy: 0.5263 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00127: saving model to training_1\cp.ckpt
Epoch 128/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5310 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00128: saving model to training_1\cp.ckpt
Epoch 129/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5331 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00129: saving model to training_1\cp.ckpt
Epoch 130/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5304 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00130: saving model to training_1\cp.ckpt
Epoch 131/150
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5325 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00131: saving model to training_1\cp.ckpt
Epoch 132/150
804/804 [=====================

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7725 - accuracy: 0.6140
Loss: 0.7725014686584473, Accuracy: 0.6139941811561584


In [30]:
# Optimization procedure #3, # Change activation function to tanh
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [31]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [32]:
# Determine which values to replace if counts are less than 1000
replace_application = list(application_type_count[application_type_count < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Determine which values to replace if counts are less than 2500
replace_class = list(CLASSIFICATION_count[CLASSIFICATION_count < 2500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [34]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [36]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [37]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                3120      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 31        
Total params: 5,581
Trainable params: 5,581
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Create a callback that saves the model's weights
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [41]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,validation_data=(X_test, y_test),callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6953 - accuracy: 0.5117 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5336 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5339 - val_loss: 0.6933 - val_accuracy: 0.5332

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5316 - val_loss: 0.6940 - val_accuracy: 0.5332

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5330 - val_loss: 0.6912 - val_accuracy: 0.5332

Epoch 00005: saving model to training_1\cp.ckpt
Epoch 6/100
804/804 [===================

804/804 [==============================] - 1s 1ms/step - loss: 0.6920 - accuracy: 0.5298 - val_loss: 0.6929 - val_accuracy: 0.5332

Epoch 00043: saving model to training_1\cp.ckpt
Epoch 44/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5306 - val_loss: 0.6920 - val_accuracy: 0.5332

Epoch 00044: saving model to training_1\cp.ckpt
Epoch 45/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5298 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00045: saving model to training_1\cp.ckpt
Epoch 46/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5335 - val_loss: 0.6930 - val_accuracy: 0.5332

Epoch 00046: saving model to training_1\cp.ckpt
Epoch 47/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6923 - accuracy: 0.5282 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00047: saving model to training_1\cp.ckpt
Epoch 48/100
804/804 [==========================


Epoch 00085: saving model to training_1\cp.ckpt
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5274 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00086: saving model to training_1\cp.ckpt
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6923 - accuracy: 0.5265 - val_loss: 0.6910 - val_accuracy: 0.5332

Epoch 00087: saving model to training_1\cp.ckpt
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5365 - val_loss: 0.6915 - val_accuracy: 0.5332

Epoch 00088: saving model to training_1\cp.ckpt
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6920 - accuracy: 0.5238 - val_loss: 0.6909 - val_accuracy: 0.5332

Epoch 00089: saving model to training_1\cp.ckpt
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5341 - val_loss: 0.6911 - val_accuracy: 0.5332

Epoch 00090: saving model to train

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7125 - accuracy: 0.5006
Loss: 0.7124904990196228, Accuracy: 0.500641405582428


In [43]:
nn.save('AlphabetSoupCharity_Optimzation.h5')